# 1. Import and Install Dependencies

In [2]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [116]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [117]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [118]:
def mediapipe_detection(image, model):
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [119]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [120]:
# Drawing functions with updated references
def draw_styled_landmarks(image, results):
    # Draw face connections if face landmarks exist
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp.solutions.face_mesh.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
    # Draw pose connections
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
    # Draw left hand connections
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
    # Draw right hand connections  
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

# 3. Extract Keypoint Values

In [121]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

# 4. Setup Folders for Collection

In [122]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Videos')

# Actions that we try to detect
actions = np.array(['absent', 'accept', 'accident', 'adult', 'aeroplane', 'after'])


# 5. Collect Keypoint Values for Training and Testing

In [124]:
max_frames_length=0
# Iterate over each action folder
for action in actions:
    # List all video files in the action folder
    video_files = [f for f in os.listdir(os.path.join(DATA_PATH, action)) if f.endswith('.mp4')]
    
    for video_file in video_files:
        cap = cv2.VideoCapture(os.path.join(DATA_PATH, action, video_file))
        sequence = int(video_file.split('.')[0])  # Assuming the file name is like '1.mp4'
        frame_num = 0

        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                # draw_styled_landmarks(image, results)

                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                os.makedirs(os.path.dirname(npy_path), exist_ok=True)
                np.save(npy_path, keypoints)

                frame_num += 1
                
        max_frames_length=max(max_frames_length,frame_num)
                # Show to screen (optional)
                # cv2.imshow('OpenCV Feed', image)

                # if cv2.waitKey(10) & 0xFF == ord('q'):
                #     break

        cap.release()
    cv2.destroyAllWindows()

In [125]:
keypoints = np.load('Videos\\absent\\1\\2.npy')
print(keypoints.shape)
print(keypoints)  # Inspect the keypoints array

# If you want to visualize it
pose_keypoints = keypoints[:132].reshape((33, 4))  # The first 132 elements correspond to pose landmarks
print(pose_keypoints)


(258,)
[ 0.52821928  0.28116015 -1.4501946   0.99957293  0.56584311  0.25630674
 -1.39514124  0.99897105  0.58841044  0.25722292 -1.3954668   0.99897707
  0.60738432  0.25837347 -1.39603341  0.9987554   0.49415317  0.25571114
 -1.38790512  0.99911529  0.4731065   0.25652716 -1.38824785  0.99919254
  0.45673651  0.25783455 -1.38870692  0.99902803  0.64055902  0.27680975
 -1.0383358   0.99906111  0.43493924  0.27848563 -0.99055099  0.99885619
  0.57318115  0.31707582 -1.31331027  0.99983442  0.48715311  0.31780311
 -1.29996932  0.99981493  0.7994231   0.46384424 -0.78672969  0.99891359
  0.28859106  0.45927426 -0.74681473  0.99908829  0.9003855   0.67853224
 -0.51077592  0.98752135  0.24780397  0.6707195  -0.52792734  0.96618944
  0.93985927  0.8846435  -0.58111334  0.86356193  0.2598314   0.87626553
 -0.70837343  0.86517024  0.9731546   0.94236642 -0.63424283  0.76467091
  0.22149174  0.92789537 -0.76138937  0.78468335  0.91936743  0.95093441
 -0.72597617  0.78645223  0.23830295  0.9299

# 6. Preprocess Data and Create Labels and Features

In [126]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [127]:
label_map = {label:num for num, label in enumerate(actions)}

In [128]:
label_map

{'absent': 0,
 'accept': 1,
 'accident': 2,
 'adult': 3,
 'aeroplane': 4,
 'after': 5}

In [129]:
sequences, labels = [], []
for action in actions:
    sequence_folders = [f for f in os.listdir(os.path.join(DATA_PATH, action)) if f.isdigit()]
    sequence_folders = np.array(sequence_folders).astype(int)
    for sequence in sequence_folders:
        window = []
        for frame_num in range(len(os.listdir(os.path.join(DATA_PATH, action,str(sequence))))):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [130]:
mxx_frames=0
for i in range(len(sequences)):
    mxx_frames=max(mxx_frames,np.array(sequences[i] ).shape[0])
print(mxx_frames)

for i in range(len(sequences)):
    seq_length = np.array(sequences[i] ).shape[0]
    if seq_length < mxx_frames:
        # Pad the sequence with zeros
        pad_length = mxx_frames - seq_length
        padding = np.zeros((pad_length, np.array(sequences[i] ).shape[1]))  # Assuming sequences[i] has shape (length, features)
        sequences[i] = np.concatenate((sequences[i], padding), axis=0)

129


In [131]:
np.array(sequences).shape

(131, 129, 258)

In [132]:
np.array(labels).shape

(131,)

In [133]:
X = np.array(sequences)

In [134]:
X.shape

(131, 129, 258)

In [135]:
y = to_categorical(labels).astype(int)

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [137]:
y_test.shape

(7, 6)

# 7. Build and Train LSTM Neural Network

In [138]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [139]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [171]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(129,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [180]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
model = Sequential()

# LSTM Layers with reduced units
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(129, 258)))
model.add(BatchNormalization())
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(BatchNormalization())

# Dense Layers with Dropout and L2 Regularization
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(actions.shape[0], activation='softmax'))

In [194]:
from tensorflow.keras.optimizers import Adam
learning_rate = 0.55  # You can adjust this value

# Update the optimizer with the custom learning rate
optimizer = Adam(learning_rate=learning_rate)

In [195]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50, callbacks=[tb_callback])

Epoch 1/50


In [193]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                       │ (None, 129, 32)             │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 129, 32)             │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 64)                  │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 206,324 (805.96 KB)

 Trainable params: 68,710 (268.40 KB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 137,422 (536.81 KB)